<a href="https://colab.research.google.com/github/RicardoHernandezRodriguez/Hackated/blob/main/WebScrapingCada_X_tiempo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import threading
import time
import schedule
from datetime import datetime, timedelta
import json
import smtplib
from email.mime.text import MimeText
from email.mime.multipart import MimeMultipart
import sqlite3
import logging
from typing import List, Dict, Optional
import requests
from dataclasses import dataclass, asdict
import os
from pathlib import Path

# Configurar logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('monitoreo_noticias.log'),
        logging.StreamHandler()
    ]
)

@dataclass
class ConfiguracionAlerta:
    """Configuración para las alertas"""
    email_smtp_server: str = "smtp.gmail.com"
    email_smtp_port: int = 587
    email_usuario: str = ""
    email_password: str = ""
    email_destinatarios: List[str] = None
    webhook_discord: str = ""
    webhook_slack: str = ""
    umbral_relevancia: int = 7  # Solo alertar si relevancia >= 7
    intervalo_monitoreo_minutos: int = 60  # Cada hora
    max_alertas_por_ciclo: int = 5

class MonitorNoticias:
    def __init__(self, config: ConfiguracionAlerta, buscador_noticias, scraper_periodicos):
        self.config = config
        self.buscador = buscador_noticias
        self.scraper = scraper_periodicos
        self.ejecutandose = False
        self.thread_monitoreo = None
        self.ultima_revision = datetime.now() - timedelta(hours=1)
        self.alertas_enviadas_hoy = 0
        self.max_alertas_diarias = 20

        # Base de datos para tracking de alertas
        self.init_db_alertas()

        # Cargar configuración desde archivo si existe
        self.cargar_configuracion()

    def init_db_alertas(self):
        """Inicializa base de datos para tracking de alertas"""
        self.conn_alertas = sqlite3.connect('alertas_monitor.db', check_same_thread=False)
        cursor = self.conn_alertas.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS alertas_enviadas (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                url_noticia TEXT UNIQUE,
                titulo TEXT,
                fecha_alerta TEXT,
                tipo_alerta TEXT,
                relevancia INTEGER
            )
        ''')
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS estadisticas_monitoreo (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                fecha TEXT,
                noticias_encontradas INTEGER,
                alertas_enviadas INTEGER,
                tiempo_ejecucion REAL,
                errores TEXT
            )
        ''')
        self.conn_alertas.commit()

    def guardar_configuracion(self):
        """Guarda configuración en archivo JSON"""
        config_dict = asdict(self.config)
        with open('config_monitoreo.json', 'w', encoding='utf-8') as f:
            json.dump(config_dict, f, indent=2, ensure_ascii=False)
        logging.info("✅ Configuración guardada en config_monitoreo.json")

    def cargar_configuracion(self):
        """Carga configuración desde archivo JSON"""
        try:
            if os.path.exists('config_monitoreo.json'):
                with open('config_monitoreo.json', 'r', encoding='utf-8') as f:
                    config_dict = json.load(f)
                    # Actualizar configuración actual
                    for key, value in config_dict.items():
                        if hasattr(self.config, key):
                            setattr(self.config, key, value)
                logging.info("✅ Configuración cargada desde archivo")
        except Exception as e:
            logging.error(f"❌ Error cargando configuración: {e}")

    def configurar_alertas_email(self, smtp_server: str, smtp_port: int,
                                usuario: str, password: str, destinatarios: List[str]):
        """Configura alertas por email"""
        self.config.email_smtp_server = smtp_server
        self.config.email_smtp_port = smtp_port
        self.config.email_usuario = usuario
        self.config.email_password = password
        self.config.email_destinatarios = destinatarios
        self.guardar_configuracion()
        logging.info("✅ Configuración de email actualizada")

    def configurar_webhook(self, discord_url: str = "", slack_url: str = ""):
        """Configura webhooks para Discord/Slack"""
        if discord_url:
            self.config.webhook_discord = discord_url
        if slack_url:
            self.config.webhook_slack = slack_url
        self.guardar_configuracion()
        logging.info("✅ Webhooks configurados")

    def enviar_alerta_email(self, noticias: List[Dict]) -> bool:
        """Envía alerta por email"""
        if not self.config.email_usuario or not self.config.email_destinatarios:
            logging.warning("⚠️ Email no configurado")
            return False

        try:
            # Crear mensaje
            msg = MimeMultipart()
            msg['From'] = self.config.email_usuario
            msg['To'] = ', '.join(self.config.email_destinatarios)
            msg['Subject'] = f"🚨 ALERTA: {len(noticias)} noticias importantes sobre Trump y economía mexicana"

            # Crear cuerpo del email
            cuerpo_html = self._generar_email_html(noticias)
            msg.attach(MimeText(cuerpo_html, 'html', 'utf-8'))

            # Enviar email
            server = smtplib.SMTP(self.config.email_smtp_server, self.config.email_smtp_port)
            server.starttls()
            server.login(self.config.email_usuario, self.config.email_password)
            text = msg.as_string()
            server.sendmail(self.config.email_usuario, self.config.email_destinatarios, text)
            server.quit()

            logging.info(f"✅ Email enviado a {len(self.config.email_destinatarios)} destinatarios")
            return True

        except Exception as e:
            logging.error(f"❌ Error enviando email: {e}")
            return False

    def enviar_alerta_discord(self, noticias: List[Dict]) -> bool:
        """Envía alerta a Discord via webhook"""
        if not self.config.webhook_discord:
            return False

        try:
            # Crear embed para Discord
            embeds = []
            for noticia in noticias[:3]:  # Máximo 3 noticias por mensaje
                embed = {
                    "title": noticia['titulo'][:256],
                    "description": noticia.get('descripcion', '')[:2048],
                    "url": noticia['url'],
                    "color": 0xFF0000 if noticia.get('relevancia', 0) >= 9 else 0xFFA500,
                    "fields": [
                        {"name": "Fuente", "value": noticia['fuente'], "inline": True},
                        {"name": "Relevancia", "value": f"{noticia.get('relevancia', 0)}/10", "inline": True},
                        {"name": "Categoría", "value": noticia.get('categoria', 'N/A'), "inline": True}
                    ],
                    "timestamp": datetime.now().isoformat()
                }
                embeds.append(embed)

            payload = {
                "content": f"🚨 **ALERTA DE NOTICIAS** - {len(noticias)} noticias importantes encontradas",
                "embeds": embeds
            }

            response = requests.post(self.config.webhook_discord, json=payload)
            response.raise_for_status()

            logging.info("✅ Alerta enviada a Discord")
            return True

        except Exception as e:
            logging.error(f"❌ Error enviando a Discord: {e}")
            return False

    def enviar_alerta_slack(self, noticias: List[Dict]) -> bool:
        """Envía alerta a Slack via webhook"""
        if not self.config.webhook_slack:
            return False

        try:
            # Crear mensaje para Slack
            texto_noticias = []
            for noticia in noticias[:5]:
                relevancia_emoji = "🔥" if noticia.get('relevancia', 0) >= 9 else "⚡"
                texto = f"{relevancia_emoji} *{noticia['titulo']}*\n📰 {noticia['fuente']} | Relevancia: {noticia.get('relevancia', 0)}/10\n🔗 <{noticia['url']}|Leer más>\n"
                texto_noticias.append(texto)

            payload = {
                "text": f"🚨 *ALERTA DE NOTICIAS* - {len(noticias)} noticias importantes",
                "blocks": [
                    {
                        "type": "section",
                        "text": {
                            "type": "mrkdwn",
                            "text": f"🚨 *ALERTA DE NOTICIAS*\n\nSe encontraron *{len(noticias)} noticias importantes* sobre Trump y la economía mexicana:\n\n" + "\n".join(texto_noticias)
                        }
                    }
                ]
            }

            response = requests.post(self.config.webhook_slack, json=payload)
            response.raise_for_status()

            logging.info("✅ Alerta enviada a Slack")
            return True

        except Exception as e:
            logging.error(f"❌ Error enviando a Slack: {e}")
            return False

    def _generar_email_html(self, noticias: List[Dict]) -> str:
        """Genera HTML para el email de alerta"""
        html = f"""
        <html>
        <head>
            <style>
                body {{ font-family: Arial, sans-serif; line-height: 1.6; color: #333; }}
                .header {{ background-color: #d32f2f; color: white; padding: 20px; text-align: center; }}
                .noticia {{ border-left: 4px solid #2196F3; padding: 15px; margin: 10px 0; background-color: #f9f9f9; }}
                .relevancia-alta {{ border-left-color: #d32f2f; }}
                .relevancia-media {{ border-left-color: #ff9800; }}
                .titulo {{ font-size: 18px; font-weight: bold; color: #1976D2; margin-bottom: 8px; }}
                .fuente {{ font-size: 12px; color: #666; }}
                .descripcion {{ margin: 10px 0; }}
                .footer {{ text-align: center; margin-top: 30px; padding: 20px; background-color: #f5f5f5; }}
            </style>
        </head>
        <body>
            <div class="header">
                <h1>🚨 ALERTA DE NOTICIAS</h1>
                <p>Monitoreo automático - {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}</p>
            </div>

            <p>Se encontraron <strong>{len(noticias)} noticias importantes</strong> sobre Trump y la economía mexicana:</p>
        """

        for noticia in noticias:
            relevancia = noticia.get('relevancia', 0)
            clase_relevancia = 'relevancia-alta' if relevancia >= 8 else 'relevancia-media' if relevancia >= 6 else ''

            html += f"""
            <div class="noticia {clase_relevancia}">
                <div class="titulo">{noticia['titulo']}</div>
                <div class="fuente">📰 {noticia['fuente']} | ⭐ Relevancia: {relevancia}/10 | 📅 {noticia.get('fecha_publicacion', 'N/A')}</div>
                <div class="descripcion">{noticia.get('descripcion', 'Sin descripción disponible')}</div>
                <p><a href="{noticia['url']}" style="color: #1976D2;">🔗 Leer noticia completa</a></p>
            </div>
            """

        html += """
            <div class="footer">
                <p>Este es un mensaje automático del sistema de monitoreo de noticias.</p>
                <p>Sistema desarrollado para seguimiento de impacto económico de políticas estadounidenses en México.</p>
            </div>
        </body>
        </html>
        """

        return html

    def ejecutar_ciclo_monitoreo(self):
        """Ejecuta un ciclo completo de monitoreo"""
        inicio_ciclo = time.time()
        errores = []

        try:
            logging.info("🔄 Iniciando ciclo de monitoreo...")

            # 1. Buscar noticias nuevas
            noticias_nuevas = []

            # Buscar con APIs
            try:
                noticias_api = self.buscador.buscar_todas_las_fuentes(dias_atras=1)
                noticias_nuevas.extend(noticias_api)
            except Exception as e:
                error_msg = f"Error en APIs: {e}"
                errores.append(error_msg)
                logging.error(f"❌ {error_msg}")

            # Buscar en periódicos específicos
            try:
                noticias_scraper = self.scraper.buscar_en_todos_los_periodicos()
                noticias_nuevas.extend(noticias_scraper)
            except Exception as e:
                error_msg = f"Error en scraping: {e}"
                errores.append(error_msg)
                logging.error(f"❌ {error_msg}")

            # 2. Filtrar noticias realmente nuevas y relevantes
            noticias_alertar = self._filtrar_noticias_para_alerta(noticias_nuevas)

            # 3. Enviar alertas si hay noticias importantes
            alertas_enviadas = 0
            if noticias_alertar and self.alertas_enviadas_hoy < self.max_alertas_diarias:
                logging.info(f"📢 Enviando alertas para {len(noticias_alertar)} noticias importantes")

                # Enviar por todos los canales configurados
                if self.enviar_alerta_email(noticias_alertar):
                    alertas_enviadas += 1

                if self.enviar_alerta_discord(noticias_alertar):
                    alertas_enviadas += 1

                if self.enviar_alerta_slack(noticias_alertar):
                    alertas_enviadas += 1

                # Marcar noticias como alertadas
                self._marcar_noticias_alertadas(noticias_alertar)
                self.alertas_enviadas_hoy += 1

            # 4. Guardar estadísticas
            tiempo_ejecucion = time.time() - inicio_ciclo
            self._guardar_estadisticas(len(noticias_nuevas), alertas_enviadas, tiempo_ejecucion, errores)

            self.ultima_revision = datetime.now()
            logging.info(f"✅ Ciclo completado: {len(noticias_nuevas)} noticias, {len(noticias_alertar)} alertas, {tiempo_ejecucion:.2f}s")

        except Exception as e:
            logging.error(f"❌ Error crítico en ciclo de monitoreo: {e}")

    def _filtrar_noticias_para_alerta(self, noticias: List[Dict]) -> List[Dict]:
        """Filtra noticias que ameritan alerta"""
        noticias_alertar = []
        cursor = self.conn_alertas.cursor()

        for noticia in noticias:
            # Verificar relevancia
            relevancia = noticia.get('relevancia', 0)
            if relevancia < self.config.umbral_relevancia:
                continue

            # Verificar si ya se envió alerta para esta noticia
            cursor.execute('SELECT id FROM alertas_enviadas WHERE url_noticia = ?', (noticia['url'],))
            if cursor.fetchone():
                continue  # Ya se alertó sobre esta noticia

            noticias_alertar.append(noticia)

            # Limitar número de alertas por ciclo
            if len(noticias_alertar) >= self.config.max_alertas_por_ciclo:
                break

        return noticias_alertar

    def _marcar_noticias_alertadas(self, noticias: List[Dict]):
        """Marca noticias como ya alertadas"""
        cursor = self.conn_alertas.cursor()
        for noticia in noticias:
            cursor.execute('''
                INSERT OR IGNORE INTO alertas_enviadas
                (url_noticia, titulo, fecha_alerta, tipo_alerta, relevancia)
                VALUES (?, ?, ?, ?, ?)
            ''', (
                noticia['url'],
                noticia['titulo'],
                datetime.now().isoformat(),
                'automatica',
                noticia.get('relevancia', 0)
            ))
        self.conn_alertas.commit()

    def _guardar_estadisticas(self, noticias_encontradas: int, alertas_enviadas: int,
                            tiempo_ejecucion: float, errores: List[str]):
        """Guarda estadísticas del ciclo de monitoreo"""
        cursor = self.conn_alertas.cursor()
        cursor.execute('''
            INSERT INTO estadisticas_monitoreo
            (fecha, noticias_encontradas, alertas_enviadas, tiempo_ejecucion, errores)
            VALUES (?, ?, ?, ?, ?)
        ''', (
            datetime.now().isoformat(),
            noticias_encontradas,
            alertas_enviadas,
            tiempo_ejecucion,
            json.dumps(errores) if errores else None
        ))
        self.conn_alertas.commit()

    def iniciar_monitoreo(self):
        """Inicia el monitoreo automático"""
        if self.ejecutandose:
            logging.warning("⚠️ El monitoreo ya está ejecutándose")
            return

        self.ejecutandose = True
        logging.info(f"🚀 Iniciando monitoreo automático cada {self.config.intervalo_monitoreo_minutos} minutos")

        # Configurar schedule
        schedule.every(self.config.intervalo_monitoreo_minutos).minutes.do(self.ejecutar_ciclo_monitoreo)

        # Ejecutar primer ciclo inmediatamente
        self.ejecutar_ciclo_monitoreo()

        # Crear thread para el loop de monitoreo
        def loop_monitoreo():
            while self.ejecutandose:
                schedule.run_pending()
                time.sleep(60)  # Verificar cada minuto

        self.thread_monitoreo = threading.Thread(target=loop_monitoreo, daemon=True)
        self.thread_monitoreo.start()

        logging.info("✅ Monitoreo iniciado correctamente")

    def detener_monitoreo(self):
        """Detiene el monitoreo automático"""
        self.ejecutandose = False
        schedule.clear()
        logging.info("🛑 Monitoreo detenido")

    def obtener_estadisticas(self) -> Dict:
        """Obtiene estadísticas del monitoreo"""
        cursor = self.conn_alertas.cursor()

        # Estadísticas generales
        cursor.execute('''
            SELECT
                COUNT(*) as total_ciclos,
                AVG(noticias_encontradas) as promedio_noticias,
                SUM(alertas_enviadas) as total_alertas,
                AVG(tiempo_ejecucion) as tiempo_promedio
            FROM estadisticas_monitoreo
            WHERE DATE(fecha) >= DATE('now', '-7 days')
        ''')
        stats_generales = cursor.fetchone()

        # Alertas por día (últimos 7 días)
        cursor.execute('''
            SELECT DATE(fecha_alerta) as fecha, COUNT(*) as cantidad
            FROM alertas_enviadas
            WHERE DATE(fecha_alerta) >= DATE('now', '-7 days')
            GROUP BY DATE(fecha_alerta)
            ORDER BY fecha DESC
        ''')
        alertas_por_dia = cursor.fetchall()

        return {
            'total_ciclos': stats_generales[0] or 0,
            'promedio_noticias_por_ciclo': round(stats_generales[1] or 0, 2),
            'total_alertas_enviadas': stats_generales[2] or 0,
            'tiempo_promedio_ciclo': round(stats_generales[3] or 0, 2),
            'alertas_por_dia': dict(alertas_por_dia),
            'ultima_revision': self.ultima_revision.strftime('%d/%m/%Y %H:%M:%S'),
            'estado': 'Ejecutándose' if self.ejecutandose else 'Detenido'
        }

    def generar_reporte_monitoreo(self) -> str:
        """Genera reporte detallado del monitoreo"""
        stats = self.obtener_estadisticas()

        reporte = f"""
📊 REPORTE DE MONITOREO AUTOMÁTICO
{'=' * 50}

🔄 Estado: {stats['estado']}
📅 Última revisión: {stats['ultima_revision']}
🔢 Total de ciclos (7 días): {stats['total_ciclos']}
📰 Promedio noticias por ciclo: {stats['promedio_noticias_por_ciclo']}
🚨 Total alertas enviadas: {stats['total_alertas_enviadas']}
⏱️  Tiempo promedio por ciclo: {stats['tiempo_promedio_ciclo']}s

📈 ALERTAS POR DÍA:
{'-' * 30}
"""

        for fecha, cantidad in stats['alertas_por_dia'].items():
            reporte += f"{fecha}: {cantidad} alertas\n"

        return reporte

    def cerrar_conexiones(self):
        """Cierra todas las conexiones"""
        if hasattr(self, 'conn_alertas'):
            self.conn_alertas.close()

# Ejemplo de uso completo
if __name__ == "__main__":
    # Importar las clases principales (asumiendo que están en el mismo directorio)
    from sistema_busqueda_noticias import BuscadorNoticias
    from scraping_periodicos_especificos import ScrapingPeriodicos

    # Configurar sistema
    config = ConfiguracionAlerta(
        email_usuario="tu_email@gmail.com",
        email_password="tu_password_app",  # Usar contraseña de aplicación
        email_destinatarios=["destino1@email.com", "destino2@email.com"],
        webhook_discord="https://discord.com/api/webhooks/...",
        umbral_relevancia=7,
        intervalo_monitoreo_minutos=60,
        max_alertas_por_ciclo=3
    )

    # Crear instancias
    buscador = BuscadorNoticias()  # news_api_key opcional
    scraper = ScrapingPeriodicos()
    monitor = MonitorNoticias(config, buscador, scraper)

    try:
        print("🚀 SISTEMA DE MONITOREO AUTOMÁTICO")
        print("=" * 50)

        # Opción 1: Ejecutar un ciclo de prueba
        print("\n1. Ejecutando ciclo de prueba...")
        monitor.ejecutar_ciclo_monitoreo()

        # Opción 2: Iniciar monitoreo continuo
        print("\n2. Iniciando monitoreo continuo...")
        monitor.iniciar_monitoreo()

        # Mantener el programa ejecutándose
        print("\n✅ Monitoreo activo. Presiona Ctrl+C para detener.")
        print(f"📊 Revisando cada {config.intervalo_monitoreo_minutos} minutos")
        print(f"🎯 Umbral de relevancia: {config.umbral_relevancia}/10")

        while True:
            time.sleep(300)  # Mostrar estadísticas cada 5 minutos
            print(f"\n📈 {datetime.now().strftime('%H:%M:%S')} - Estadísticas:")
            stats = monitor.obtener_estadisticas()
            print(f"   Estado: {stats['estado']}")
            print(f"   Última revisión: {stats['ultima_revision']}")
            print(f"   Total alertas: {stats['total_alertas_enviadas']}")

    except KeyboardInterrupt:
        print("\n🛑 Deteniendo monitoreo...")
        monitor.detener_monitoreo()
        monitor.cerrar_conexiones()
        print("✅ Monitoreo detenido correctamente")